In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup as bs

import time
import math
import pandas as pd
import pickle

from tqdm import trange, tqdm

In [10]:
driver = webdriver.Chrome()
driver.maximize_window()
root = 'https://www.fundguide.net/Fund/SimpleSearch'
driver.get(root)

In [11]:
# 투자유형 - 국내주식형 체크
wait = WebDriverWait(driver, 1)
domestic_stock_label = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[@for='chkListA0']")))
domestic_stock_label.click()

In [12]:
 # 검색 버튼 클릭
search_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn--basic2']")))
search_button.click()
time.sleep(2)

In [13]:
# 맨 마지막(462페이지)으로 가기
search_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='tabPaging']/div/tr/button[13]")))
search_button.click()
time.sleep(2)

In [14]:
for _ in trange(462):
    # 현재 페이지 번호 : 462 ~ 1
    page_num = driver.find_element(By.TAG_NAME, 'em').text

    # 펀드명 리스트
    fund_details = driver.find_elements(By.CSS_SELECTOR, "a.fund_detail")
    #     # 각 <a> 태그에서 텍스트 추출
    fund_names = [fund_detail.text for fund_detail in fund_details]
    # fund_names
    
    for i in range(1,len(fund_names)+1):
        flag_rtns = True
        # print(f'{page_num}페이지 {i}번째 펀드 : {fund_names[i-1]}')

        # 첫 번째 펀드명 클릭
        a = f"//*[@id='resultList']/tr[{i}]/td[2]/a/span"
        # print(a)
        first_fund_detail = wait.until(EC.element_to_be_clickable((By.XPATH, a)))
        first_fund_detail.click()
        # print(i)

        # '성과' 버튼이 포함된 <li> 요소 찾기 및 클릭
        performance_li = wait.until(EC.element_to_be_clickable((By.XPATH, "//li[button[@class='btn--tab__fund' and @value='1']]")))
        performance_button = performance_li.find_element(By.XPATH, ".//button")
        performance_button.click()

        # 코로나 버튼
        corona_button = driver.find_element(By.XPATH, '//*[@id="btnRatetime"]/button[2]/span')
        corona_button.click()
        time.sleep(3)
        
        # 3년치 버튼
        three_button = driver.find_element(By.XPATH, '//*[@id="radio_rate"]/span[5]/label/span')
        three_button.click()
        time.sleep(3)
        
        # 수익률 마지막 페이지로 이동
        try:
            # 마지막 으로 가는 버튼
            last_button = driver.find_element(By.XPATH, '//*[@id="tabPaging_info"]/div/tr/button[13]')
            last_button.click()

        except:
            try:
                # 페이지 수가 10 미만인 경우
                html_data = driver.page_source
                soup = bs(html_data, 'html.parser')
                btns = soup.find_all('button', attrs={'class' : 'btn--cur'})
                page_cnt = int(btns[-1].text)

                last_button = driver.find_element(By.XPATH, f'//*[@id="tabPaging_info"]/div/tr/button[{page_cnt+3}]')
                last_button.click()
                # print("페이지 번호 10 미만")
            except:
                print(f'{page_num}페이지 {i}번째 펀드 : {fund_names[i-1]}', ": 코로나 데이터 없음")
                flag_rtns = False # x누르고 다음 펀드로
        time.sleep(3)

        """수익률 데이터 추출"""
        if flag_rtns==True:
            table_data = []
            rtn_page = int(driver.find_element(By.XPATH, '//*[@id="tabPaging_info"]/div/tr/em').text) # 마지막 페이지 번호

            for j in trange(rtn_page):
                time.sleep(1)
                table = driver.find_element(By.ID, 'tbl_rate')
                headers = [header.text for header in table.find_elements(By.XPATH, ".//thead//th")]
                rows = table.find_elements(By.XPATH, ".//tbody//tr")

                for row in rows:
                    cells = row.find_elements(By.XPATH, ".//td")
                    cell_data = [cell.text for cell in cells]
                    table_data.append(cell_data)
                
                # 이전 페이지로 이동
                prev_button = driver.find_element(By.XPATH, '//*[@id="tabPaging_info"]/div/tr/button[2]')
                prev_button.click()

            # 데이터 출력
            df = pd.DataFrame(table_data, columns=headers)
            df.drop_duplicates()

            file_name =fund_names[i-1]
            df.to_csv(f'data/{file_name}.csv', index=False)

        # 펀드 상세 창 닫기
        end = wait.until(EC.element_to_be_clickable((By.ID, 'btnClose')))
        end.click()
    
    # 이전 페이지로 가기
    search_button = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//*[@id='tabPaging']/div/tr/button[2]")
    ))
    search_button.click()  
    time.sleep(2)

100%|██████████| 7/7 [00:14<00:00,  2.12s/it]


461페이지 3번째 펀드 : 우리WOORI반도체밸류체인액티브증권상장지수투자신탁(주식) : 코로나 데이터 없음
461페이지 4번째 펀드 : 우리나라신성장TOP20증권자투자신탁 1(주식)ClassC : 코로나 데이터 없음


100%|██████████| 72/72 [02:27<00:00,  2.05s/it]


461페이지 6번째 펀드 : 삼성코스닥150인덱스증권투자신탁[주식-파생형]_C-Pe : 코로나 데이터 없음
461페이지 7번째 펀드 : 우리나라신성장TOP20증권자투자신탁 1(주식)ClassS-P(퇴직) : 코로나 데이터 없음
461페이지 8번째 펀드 : BNK삼성전자중소형증권투자신탁 1(주식)ClassS-P : 코로나 데이터 없음
461페이지 9번째 펀드 : 하나K-ing증권투자신탁[주식]ClassC-P : 코로나 데이터 없음
461페이지 10번째 펀드 : 우리중소형고배당증권자투자신탁 1(주식)ClassS-P(퇴직) : 코로나 데이터 없음


100%|██████████| 7/7 [00:10<00:00,  1.46s/it]


...

443페이지 9번째 펀드 : 우리WOORI반도체밸류체인액티브증권상장지수투자신탁(주식) : 코로나 데이터 없음
443페이지 10번째 펀드 : 우리나라신성장TOP20증권자투자신탁 1(주식)ClassC : 코로나 데이터 없음


100%|██████████| 113/113 [02:48<00:00,  1.49s/it]


442페이지 7번째 펀드 : 신한SOL2차전지소부장Fn증권상장지수투자신탁[주식] : 코로나 데이터 없음
442페이지 8번째 펀드 : 미래에셋TIGER2차전지소재Fn증권상장지수투자신탁(주식) : 코로나 데이터 없음


  5%|▍         | 22/462 [3:26:10<68:43:28, 562.29s/it]  


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.182); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6D38DEEB2+31554]
	(No symbol) [0x00007FF6D3857EE9]
	(No symbol) [0x00007FF6D371872A]
	(No symbol) [0x00007FF6D371E55E]
	(No symbol) [0x00007FF6D3720EB1]
	(No symbol) [0x00007FF6D3720F50]
	(No symbol) [0x00007FF6D37621DB]
	(No symbol) [0x00007FF6D378D02A]
	(No symbol) [0x00007FF6D375BA76]
	(No symbol) [0x00007FF6D378D240]
	(No symbol) [0x00007FF6D37AC977]
	(No symbol) [0x00007FF6D378CDD3]
	(No symbol) [0x00007FF6D375A33B]
	(No symbol) [0x00007FF6D375AED1]
	GetHandleVerifier [0x00007FF6D3BE8B2D+3217341]
	GetHandleVerifier [0x00007FF6D3C35AF3+3532675]
	GetHandleVerifier [0x00007FF6D3C2B0F0+3489152]
	GetHandleVerifier [0x00007FF6D398E786+750614]
	(No

수정전 코드

In [ ]:
# for i in range(1,462):
#     # 이전 페이지로 가기
#     search_button = wait.until(EC.element_to_be_clickable(
#         (By.XPATH, "//*[@id='tabPaging']/div/tr/button[2]")
#     ))
#     search_button.click()  
#     time.sleep(2)

#     fund_details = driver.find_elements(By.CSS_SELECTOR, "a.fund_detail")
#     #     # 각 <a> 태그에서 텍스트 추출
#     fund_names = [fund_detail.text for fund_detail in fund_details]
#     fund_names
    
#     for i in range(1,len(fund_names)+1):
#         print(i)
#         # 첫 번째 펀드명 클릭
#         a = "//*[@id='resultList']/tr["+ str(i) +"]/td[2]/a/span"
#         print(a)
#         first_fund_detail = wait.until(EC.element_to_be_clickable((By.XPATH, a)))
#         first_fund_detail.click()
#         print(i)
#         # '성과' 버튼이 포함된 <li> 요소 찾기 및 클릭
#         performance_li = wait.until(EC.element_to_be_clickable((By.XPATH, "//li[button[@class='btn--tab__fund' and @value='1']]")))
#         performance_button = performance_li.find_element(By.XPATH, ".//button")
#         performance_button.click()

#         # 코로나 버튼
#         corona_button = driver.find_element(By.XPATH, '//*[@id="btnRatetime"]/button[2]/span')
#         corona_button.click()
#         time.sleep(3)
        
#         # 3년치 버튼
#         three_button = driver.find_element(By.XPATH, '//*[@id="radio_rate"]/span[5]/label/span')
#         three_button.click()
#         time.sleep(3)
        
#         # 마지막 으로 가는 버튼
#         try:
#             last_button = driver.find_element(By.XPATH, '//*[@id="tabPaging_info"]/div/tr/button[13]')
#             last_button.click()

#         except:
            

#             html_data = driver.page_source
#             soup = bs(html_data, 'html.parser')
#             btns = soup.find_all('button', attrs={'class' : 'btn--cur'})
#             page_cnt = int(btns[-1].text)

#             last_button = driver.find_element(By.XPATH, f'//*[@id="tabPaging_info"]/div/tr/button[{page_cnt+3}]')
#             last_button.click()

            
#         time.sleep(3)
            
#         table_data = []
#         for j in range(100):
#             if j < 1:
                
#                 time.sleep(1)
#                 table = driver.find_element(By.ID, 'tbl_rate')
#                 headers = [header.text for header in table.find_elements(By.XPATH, ".//thead//th")]
#                 rows = table.find_elements(By.XPATH, ".//tbody//tr")

#                 for row in rows:
#                     cells = row.find_elements(By.XPATH, ".//td")
#                     cell_data = [cell.text for cell in cells]
#                     table_data.append(cell_data)

#                 prev_button = driver.find_element(By.XPATH, '//*[@id="tabPaging_info"]/div/tr/button[2]')
#                 prev_button.click()

#                 print(j)
#             else:
        
#                 # 데이터 출력
#                 df = pd.DataFrame(table_data, columns=headers)
#                 df.drop_duplicates()

#                 file_name =fund_names[i-1]
#                 file_name          
#                 df.to_csv(file_name, index=False)
#         end = wait.until(EC.element_to_be_clickable((By.ID, 'btnClose')))
#         end.click()
#     else:
#         break